In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.bronze_etl
COMMENT 'Camada Bronze para armazenamento de dados brutos';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.silver_etl
COMMENT 'Camada Silver para armazenamento de dados tratados';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.gold_etl
COMMENT 'Camada Golf para armazenamento de dados prontos para uso';